In [1]:
import torch
import numpy as np
from skorch import NeuralNetClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from mlp import MLP

In [2]:
X = np.load('./data/lbp/features.npy').astype(np.float32) # [num, 256]
y = np.load('./data/lbp/labels.npy').astype(np.int64) # [num]

device = 'cpu'
# device = 'mps'
print("Device is set to:", device)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

Device is set to: cpu


In [3]:
print(np.isnan(X).any(), np.isinf(X).any())
print(np.isnan(y).any(), np.isinf(y).any())

False False
False False


In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

In [5]:
from skorch.callbacks import GradientNormClipping
net = NeuralNetClassifier(
    MLP,
    module__input_size=256,
    module__output_size=4,
    max_epochs=50,
    lr=0.01,  # Start with a lower learning rate
    iterator_train__shuffle=True,
    callbacks=[GradientNormClipping(0.5)],  # Add gradient clipping
)

In [6]:
param_grid = {
    'module__hidden_sizes': [[128], [256], [256, 128], [256, 128, 64]],
    'module__dropout_rate': [0.2, 0.3],
    'lr': [0.001],
    'batch_size': [32, 64]
}

In [7]:
cv = StratifiedKFold(n_splits=3, random_state=42, shuffle=True)

gs = GridSearchCV(net, param_grid, refit=True, cv=cv, scoring='accuracy', verbose=2)

gs.fit(X_train, y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1           nan       0.2706           nan  0.1933
      2           nan       0.2622           nan  0.2126
      3           nan       0.2631           nan  0.1929
      4           nan       0.2617           nan  0.1688
      5           nan       0.2582           nan  0.1793
      6           nan       0.2586           nan  0.1897
      7           nan       0.2622           nan  0.1778
      8           nan       0.2662           nan  0.1695
      9           nan       0.2693           nan  0.1681
     10           nan       0.2697           nan  0.1927
     11           nan       0.2697           nan  0.2072
     12           nan       0.2688           nan  0.1952
     13           nan       0.2693           nan  0.1841
     14           nan       0.2688           nan  0.1716
     15           nan      

GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=42, shuffle=True),
             estimator=<class 'skorch.classifier.NeuralNetClassifier'>[uninitialized](
  module=<class 'mlp.MLP'>,
  module__input_size=256,
  module__output_size=4,
),
             param_grid={'batch_size': [32, 64], 'lr': [0.001],
                         'module__dropout_rate': [0.2, 0.3],
                         'module__hidden_sizes': [[128], [256], [256, 128],
                                                  [256, 128, 64]]},
             scoring='accuracy', verbose=2)

In [8]:
print(f'Best Params: {gs.best_params_}')
print(f'Best Cross-Validation Accuracy: {gs.best_score_}')

# Evaluate on validation set
y_pred = gs.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')

Best Params: {'batch_size': 32, 'lr': 0.001, 'module__dropout_rate': 0.3, 'module__hidden_sizes': [256, 128, 64]}
Best Cross-Validation Accuracy: 0.4824001889912592
Validation Accuracy: 0.29435388613276636
